In [7]:
import mysql.connector as sql
import pandas as pd
from mysql.connector import Error

## Problem Statement 18:

In [8]:
def create_server_connection(host_name, user_name, user_password):
    connection = None
    try:
        connection = sql.connect(host=host_name, user=user_name, passwd=user_password)
        print("MySQL Database connection successful")
    except Error as err:
        print(f"The error '{err}' occurred")

    return connection

pw = "42424242"
connection = create_server_connection("localhost", "root", pw)

MySQL Database connection successful


In [9]:
def create_db(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        print("Database created successfully")
    except Error as err:
        print(f"The error '{err}' occurred")


connection = create_db(connection, "CREATE DATABASE if not exists healthcare_db")

Database created successfully


In [10]:
def create_db_connection(host_name, user_name, user_password, db_name):
    connection = None
    try:
        connection = sql.connect(host=host_name, user=user_name, passwd=user_password, database=db_name)
        print("MySQL Database connection successful")
    except Error as err:
        print(f"The error '{err}' occurred")

    return connection

connection = create_db_connection("localhost", "root", pw, "healthcare_db")

MySQL Database connection successful


In [11]:
def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Query successful")
    except Error as err:
        print(f"The error '{err}' occurred")

In [12]:
def run_query(connection, query):
    cursor = connection.cursor()
    result = None
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        columns = [column[0] for column in cursor.description]
        df = pd.DataFrame(result, columns=columns)
        return df
    except Error as err:
        print(f"The error '{err}' occurred")

In [13]:
execute_query(connection, "USE healthcare_db")

Query successful


In [14]:
execute_query(connection, """CREATE TABLE IF NOT EXISTS census (
    District VARCHAR(255),
    State_UT VARCHAR(255),
    Population INT,
    Male INT,
    Female INT,
    Literate INT,
    Literate_Male INT,
    Literate_Female INT,
    Households_Rural INT,
    Households_Urban INT,
    Households INT,
    Young_and_Adult INT,
    Middle_Aged INT,
    Senior_Citizen INT,
    Age_Not_Stated INT,
    PRIMARY KEY (District)
);""")


Query successful


In [15]:
execute_query(connection, """CREATE TABLE IF NOT EXISTS housing(
District VARCHAR(255),
    Households_Rural INT,
    Households_Rural_Livable INT,
    Households_Rural_Dilapidated INT,
    Households_Rural_Toilet_Premise INT,
    Households_Urban INT,
    Households_Urban_Livable INT,
    Households_Urban_Dilapidated INT,
    Households_Urban_Toilet_Premise INT,
    PRIMARY KEY (District),
    FOREIGN KEY (District) REFERENCES census(District)
);
""")

Query successful


In [16]:
execute_query(connection, """
              CREATE TABLE IF NOT EXISTS all_hospitals(
    State_UT VARCHAR(255),
    Number_of_Primary_Health_Centers_PHCs INT,
    Community_Health_Centers_CHCs INT,
    Sub_District_Divisional_Hospitals_SDHs INT,
    District_Hospitals_DHs INT,
    Hospitals INT,
    HospitalBeds INT,
    PRIMARY KEY (State_UT)
);
""")

Query successful


In [17]:
execute_query(connection,""" CREATE TABLE IF NOT EXISTS government_hospital (
    State_UT VARCHAR(255),
    Rural_Government_Hospitals INT,
    Rural_Government_Beds INT,
    Urban_Government_Hospitals INT,
    Urban_Government_Beds INT,
    Last_Updated DATE,
    PRIMARY KEY (State_UT),
    FOREIGN KEY (State_UT) REFERENCES all_hospitals(State_UT)
);""")

Query successful


## Problem Statement 19: 

In [18]:
run_query(connection, """SELECT *
FROM census
INNER JOIN housing ON census.District = housing.District
INNER JOIN all_hospitals ON census.`State_UT` = all_hospitals.`State_UT`
INNER JOIN government_hospital ON all_hospitals.`State_UT` = government_hospital.`State_UT`
LIMIT 3""")

,District,State_UT,Population,Male,Female,Literate,Literate_Male,Literate_Female,Households_Rural,Households_Urban,...,Sub_District_Divisional_Hospitals_SDHs,District_Hospitals_DHs,Hospitals,HospitalBeds,State_UT,Rural_Government_Hospitals,Rural_Government_Beds,Urban_Government_Hospitals,Urban_Government_Beds,Last_Updated


## Problem Statement 21

In [19]:
execute_query(connection, """
              DROP FUNCTION IF EXISTS get_population_district;
              """)

Query successful


In [20]:
execute_query(connection, """
              CREATE FUNCTION IF NOT EXISTS get_population_district(district_name TEXT)
              RETURNS INT
              DETERMINISTIC
              BEGIN
              DECLARE result INT;
              SELECT Population INTO result FROM census WHERE District = district_name;
              RETURN result;
              END;
              """)

Query successful


In [21]:
run_query(connection, "SELECT get_population_district('Ahmadabad')")

,get_population_district('Ahmadabad')
0,None


In [22]:
execute_query(connection, """
              DROP FUNCTION IF EXISTS get_population_state;
              """)

Query successful


In [23]:
execute_query(connection,"""
              CREATE FUNCTION IF NOT EXISTS get_population_state(state_name TEXT)
RETURNS INT
DETERMINISTIC
BEGIN
    DECLARE result INT;
    SELECT SUM(Population) INTO result
    FROM census
    WHERE State_UT = state_name
    GROUP BY State_UT;
    RETURN COALESCE(result, 0);
END;
              """)

Query successful


In [24]:
run_query(connection, "SELECT get_population_state('Gujarat');")

,get_population_state('Gujarat')
0,0


In [25]:
execute_query(connection, """
              DROP FUNCTION IF EXISTS senior_citizen_population;
              """)

Query successful


In [26]:
execute_query(connection, """
              CREATE FUNCTION IF NOT EXISTS senior_citizen_population(district_name TEXT)
              RETURNS INT
              DETERMINISTIC
              BEGIN
              DECLARE result INT;
              SELECT SUM(Senior_Citizen) INTO result FROM census WHERE District = district_name;
              RETURN result;
              END;
              """)

Query successful


In [27]:
run_query(connection, "SELECT senior_citizen_population('Ahmadabad')")

,senior_citizen_population('Ahmadabad')
0,None


In [28]:
execute_query(connection, """
              DROP FUNCTION IF EXISTS get_hospital_beds;
              """)

Query successful


In [29]:
execute_query(connection, """
              CREATE FUNCTION IF NOT EXISTS get_hospital_beds(state_name TEXT)
              RETURNS INT
              DETERMINISTIC
              BEGIN
              DECLARE result INT;
              SELECT HospitalBeds INTO result FROM all_hospitals WHERE State_UT = state_name;
              RETURN result;
              END;
              """)

Query successful


In [30]:
run_query(connection, "SELECT get_hospital_beds('Gujarat');")

,get_hospital_beds('Gujarat')
0,None


In [31]:
execute_query(connection, "DROP FUNCTION IF EXISTS get_govt_hospital_beds;")

Query successful


In [32]:
execute_query(connection, """
              CREATE FUNCTION IF NOT EXISTS get_govt_hospital_beds(state_name TEXT)
              RETURNS INT
              DETERMINISTIC
              BEGIN
              DECLARE result INT;
              SELECT Urban_Government_Beds+Rural_Government_Beds INTO result FROM government_hospital WHERE State_UT = state_name;
              RETURN result;
              END;
              """)

Query successful


In [33]:
run_query(connection, "SELECT get_govt_hospital_beds('Gujarat');")

,get_govt_hospital_beds('Gujarat')
0,None


In [34]:
execute_query(connection, "DROP FUNCTION IF EXISTS beds_per_lakh;")

Query successful


In [35]:
execute_query(connection, """
              CREATE FUNCTION IF NOT EXISTS beds_per_lakh(state_name TEXT)
              RETURNS FLOAT
              DETERMINISTIC
              BEGIN
              DECLARE result FLOAT;
              SELECT (get_hospital_beds(state_name)/(get_population_state(state_name))*100000) INTO result;
              RETURN result;
              END;
              """)

Query successful


In [36]:
run_query(connection, "SELECT beds_per_lakh('Gujarat');")

,beds_per_lakh('Gujarat')
0,None


In [37]:
execute_query(connection, "DROP FUNCTION IF EXISTS govt_beds_per_lakh;")

Query successful


In [38]:
execute_query(connection, """
              CREATE FUNCTION IF NOT EXISTS govt_beds_per_lakh(state_name TEXT)
              RETURNS FLOAT
              DETERMINISTIC
              BEGIN
              DECLARE result FLOAT;
              SELECT (get_govt_hospital_beds(state_name)/(get_population_state(state_name))*100000) INTO result;
              RETURN result;
              END;
              """)

Query successful


In [39]:
run_query(connection, "SELECT govt_beds_per_lakh('Gujarat');")

,govt_beds_per_lakh('Gujarat')
0,None


FileNotFoundError: [Errno 2] No such file or directory: 'Data/north_east_states.txt'